# LangChain with networkx

Date: 2024/08/09

Reference: https://python.langchain.com/v0.1/docs/integrations/graphs/networkx/

In [2]:
from langchain.indexes import GraphIndexCreator
from langchain_openai import OpenAI

In [3]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))